# Measuring Mtb growth by area

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from macrohet import visualise, dataio, tile
from tqdm.auto import tqdm
import os
import btrack
import json
import napari
import matplotlib as mpl
import matplotlib.patches as patches
import pickle
import glob
import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import subprocess
from scipy.interpolate import UnivariateSpline
import glob
scale_factor = 6048/1200
resolution_m_per_pixel = 1.4949402023919043E-07
min_mtb_area = 88 * ((resolution_m_per_pixel * 1000000) ** 2) # in um2

# set display params
sns.set(style = 'white')
# Set the default font to Helvetica
mpl.rcParams['font.family'] = 'Liberation Sans'
# Get the color palette of the PiYG color map
expanded_piyg = visualise.color_palette('expanded_piyg').colors
sns.set_palette(expanded_piyg)
# # Display the colors in the PiYG color map
# visualise.show_colors('expanded_piyg')

### Load metadata

In [2]:
base_dir = '/mnt/DATA/macrohet/macrohet_images/PS0000/'
metadata_path = os.path.join(base_dir, 'Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
metadata_fn = os.path.join(base_dir, 'Images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  

Reading metadata XML file...
Extracting metadata complete!
Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


# Load sc data

In [3]:
df = pd.read_pickle('/mnt/DATA/macrohet/results/preliminary_sc_measures/sc_dfs/sc_df_GT_70_area.pkl')
df

,Time (hours),x,y,Infection status,Initial infection status,Final infection status,Intracellular mean Mtb content,Intracellular thresholded Mtb content,Eccentricity,MSD,...,Technical replicate,Cell ID,Acquisition ID,Unique ID,dMtb,Strain/Compound,Mtb area,Mphi area,Mphi gfp,Max. dMtb gradient
0,0,457.172943,8.791715,False,False,False,343.157196,2.025801,0.929721,0.000000,...,1,426,"(3, 4)",426.3.4,NaN,RD1,NaN,20542.0,1303.328003,NaN
1,1,459.138947,6.718642,False,False,False,344.687042,0.000000,0.977788,2.857062,...,1,426,"(3, 4)",426.3.4,NaN,RD1,NaN,16683.0,1305.481812,NaN
2,2,460.555237,10.785886,False,False,False,343.875061,0.000000,0.733415,4.306780,...,1,426,"(3, 4)",426.3.4,NaN,RD1,NaN,18935.0,1356.168945,NaN
3,3,455.878815,11.143067,False,False,False,345.565796,1.825300,0.685354,4.690043,...,1,426,"(3, 4)",426.3.4,NaN,RD1,NaN,19937.0,1376.485962,NaN
4,4,462.634186,14.050420,False,False,False,350.585083,4.337127,0.727210,7.354437,...,1,426,"(3, 4)",426.3.4,NaN,RD1,NaN,21793.0,1507.209229,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296992,70,628.562500,1099.782104,False,False,False,377.731873,4.977096,0.697015,5.203749,...,2,935,"(6, 9)",935.6.9,NaN,BDQ,NaN,101991.0,1199.034180,NaN
296993,71,628.463684,1102.144409,False,False,False,377.812103,5.463210,0.690471,2.364371,...,2,935,"(6, 9)",935.6.9,NaN,BDQ,NaN,100978.0,1033.890015,NaN
296994,72,628.672058,1101.594727,False,False,False,376.812805,4.944454,0.665902,0.587853,...,2,935,"(6, 9)",935.6.9,NaN,BDQ,NaN,108397.0,1061.693726,NaN
296995,73,630.372803,1102.817749,False,False,False,377.437073,5.027092,0.647429,2.094831,...,2,935,"(6, 9)",935.6.9,NaN,BDQ,NaN,107079.0,1114.690552,NaN


### Get more mtb area measurements

In [4]:
df.sort_values(by=['Acquisition ID', 'Cell ID', 'Time (hours)'], inplace=True)

In [8]:
image_dir = os.path.join(base_dir, 'Images')
mtb_areas_dict = {}
# Create the directory if it doesn't exist
output_dir = 'sc_mtb_areas'
os.makedirs(output_dir, exist_ok=True)

In [9]:
for acq_ID in tqdm(df['Acquisition ID'].unique(), total = len(df['Acquisition ID'].unique())):
    
    # isolate row and column from acq ID
    row, column = acq_ID

    ### load tracks
    with btrack.io.HDF5FileHandler(f'/mnt/DATA/macrohet/labels/PS0000/macrohet_seg_model/{acq_ID}.h5', 
                                       'r', 
                                       obj_type='obj_type_1'
                                       ) as reader:
        masks = reader.segmentation 

    # get mtb channel image
    images = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, column, 
                                 set_plane='max_proj',
                                 set_channel=2,).compute().compute()
    
    # isolate just rows from that position 
    subset_df = df[df['Acquisition ID'] == acq_ID]

    # iterate over cells in that position
    for n, track_id in tqdm(enumerate(subset_df['Unique ID'].unique()), 
                        total = len(subset_df['Unique ID'].unique()), 
                        desc = f'Iterating over cells in position {acq_ID}', 
                        leave = False):

        # isolate just one cell
        track_data = subset_df[subset_df['Unique ID'] == track_id]
        # create empty list to store areas in 
        mtb_areas = []
        # check if mtb areas havent already been calculated 
        if track_data['Mtb area'].isna().all():
            # iterate over timepoints in that one cell
            for i, df_row in tqdm(track_data.iterrows(), 
                               total=len(track_data),
                               desc=f'Extracting mtb area over each frame: {track_id}', 
                               leave = False):
                
                # get coords
                t, y, x = df_row['Time (hours)'], int(df_row['y']), int(df_row['x'])
                y, x = int(y*scale_factor), int(x*scale_factor)
                # select proper frame
                frame = masks[t, ...]
                # check to see if mask exists
                if frame[y, x]:
                    # select cell of interest
                    mask = frame == frame[y, x]
                    # get mtb channel at set frame
                    mtb_frame = images[t, 0]
                    # only load the crop bounding box necessary
                    rows = np.any(mask, axis=1)
                    cols = np.any(mask, axis=0)
                    # Bounding box's top and bottom
                    ymin, ymax = np.where(rows)[0][[0, -1]]
                    # Bounding box's left and right
                    xmin, xmax = np.where(cols)[0][[0, -1]]
                    # The bounding box is defined by the coordinates (xmin, ymin) and (xmax, ymax)
                    bounding_box = (xmin, ymin, xmax, ymax)
                    bounded_image = mtb_frame[ymin:ymax+1, xmin:xmax+1]
                    bounded_mask = mask[ymin:ymax+1, xmin:xmax+1]
                    masked_image = bounded_image*bounded_mask 
                    thresholded_masked_image = masked_image >=480
                    mtb_area = np.sum(thresholded_masked_image)
                    # convert area from pixel to um2 
                    mtb_area = mtb_area * ((resolution_m_per_pixel * 1000000) ** 2)
                    mtb_areas.append(mtb_area)
                else:
                    mtb_areas.append(0)  # Append 0 if no mask exists
                
                mtb_areas_dict[track_id] = mtb_areas
                # Save the dictionary to a file
                with open('mtb_areas_dict.pkl', 'wb') as file:
                    pickle.dump(mtb_areas_dict, file)

    # Save each mtb_areas list to a separate file
    if mtb_areas:  # Check if the list is not empty
        filename = os.path.join(output_dir, f'mtb_areas_{track_id}.pkl')
        with open(filename, 'wb') as file:
            pickle.dump(mtb_areas, file)

  0%|          | 0/24 [00:00<?, ?it/s]

[INFO][2023/12/01 09:04:53 AM] Opening HDF file: /mnt/DATA/macrohet/labels/PS0000/macrohet_seg_model/(3, 4).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/DATA/macrohet/labels/PS0000/macrohet_seg_model/(3, 4).h5...
[INFO][2023/12/01 09:05:03 AM] Loading segmentation (75, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (75, 6048, 6048)
[INFO][2023/12/01 09:05:03 AM] Closing HDF file: /mnt/DATA/macrohet/labels/PS0000/macrohet_seg_model/(3, 4).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/DATA/macrohet/labels/PS0000/macrohet_seg_model/(3, 4).h5


Iterating over cells in position (3, 4):   0%|          | 0/178 [00:00<?, ?it/s]

Extracting mtb area over each frame: 2.3.4:   0%|          | 0/74 [00:00<?, ?it/s]

Extracting mtb area over each frame: 5.3.4:   0%|          | 0/75 [00:00<?, ?it/s]

...

Extracting mtb area over each frame: 814.6.9:   0%|          | 0/73 [00:00<?, ?it/s]

Extracting mtb area over each frame: 817.6.9:   0%|          | 0/73 [00:00<?, ?it/s]

Extracting mtb area over each frame: 819.6.9:   0%|          | 0/73 [00:00<?, ?it/s]

Extracting mtb area over each frame: 865.6.9:   0%|          | 0/72 [00:00<?, ?it/s]

Extracting mtb area over each frame: 935.6.9:   0%|          | 0/71 [00:00<?, ?it/s]

# Integrate the areas into the main dataframe

In [19]:
df.sort_values(by=['Acquisition ID', 'Cell ID', 'Time (hours)'], inplace=True)

In [20]:
df

,Time (hours),x,y,Infection status,Initial infection status,Final infection status,Intracellular mean Mtb content,Intracellular thresholded Mtb content,Eccentricity,MSD,...,Technical replicate,Cell ID,Acquisition ID,Unique ID,dMtb,Strain/Compound,Mtb area,Mphi area,Mphi gfp,Max. dMtb gradient
5619,0,879.310364,836.194092,False,False,False,384.248047,26.342258,0.491442,0.000000,...,1,2,"(3, 4)",2.3.4,NaN,RD1,NaN,36151.0,2315.506104,NaN
5620,1,879.931519,840.605713,False,False,False,378.661133,21.232080,0.707418,4.455136,...,1,2,"(3, 4)",2.3.4,NaN,RD1,NaN,41309.0,2177.926270,NaN
5621,2,887.574646,838.376038,False,False,False,375.626007,17.518320,0.461908,7.961711,...,1,2,"(3, 4)",2.3.4,NaN,RD1,NaN,46481.0,2027.962036,NaN
5622,3,886.833008,839.418884,False,False,False,378.516296,19.544024,0.722869,1.279670,...,1,2,"(3, 4)",2.3.4,NaN,RD1,NaN,33754.0,2269.419434,NaN
5623,4,891.136169,843.841553,False,False,False,378.227509,20.069654,0.612924,6.170672,...,1,2,"(3, 4)",2.3.4,NaN,RD1,NaN,42180.0,1916.314453,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296992,70,628.562500,1099.782104,False,False,False,377.731873,4.977096,0.697015,5.203749,...,2,935,"(6, 9)",935.6.9,NaN,BDQ,NaN,101991.0,1199.034180,NaN
296993,71,628.463684,1102.144409,False,False,False,377.812103,5.463210,0.690471,2.364371,...,2,935,"(6, 9)",935.6.9,NaN,BDQ,NaN,100978.0,1033.890015,NaN
296994,72,628.672058,1101.594727,False,False,False,376.812805,4.944454,0.665902,0.587853,...,2,935,"(6, 9)",935.6.9,NaN,BDQ,NaN,108397.0,1061.693726,NaN
296995,73,630.372803,1102.817749,False,False,False,377.437073,5.027092,0.647429,2.094831,...,2,935,"(6, 9)",935.6.9,NaN,BDQ,NaN,107079.0,1114.690552,NaN


In [22]:
# Iterate over the dictionary
for track_id, areas in tqdm(mtb_areas_dict.items(), total = 3888):
    # Filter the DataFrame and update 'Mtb area' for matching rows
    mask = df['Unique ID'] == track_id
    if mask.sum() == len(areas):  # Check if the lengths match
        df.loc[mask, 'Mtb area'] = areas
    else:
        print(f"Length mismatch for track ID {track_id}")

  0%|          | 0/3888 [00:00<?, ?it/s]

### Create new measuremnet for dMtb

In [39]:
# Ensure that 'Time (hours)' is sorted within each 'Unique ID'
df.sort_values(by=['Unique ID', 'Time (hours)'], inplace=True)

# Calculate the change in 'Mtb area' within each 'Unique ID'
df['dMtb/dt'] = df.groupby('Unique ID')['Mtb area'].diff()

# Optionally, handle NaN values if needed (e.g., fill with 0)
df['dMtb/dt'].fillna(0, inplace=True)

In [40]:
# Ensure that 'Time (hours)' is sorted within each 'Unique ID'
df.sort_values(by=['Unique ID', 'Time (hours)'], inplace=True)

# Calculate the percentage change in 'Mtb area' and store it in 'dMtb'
df['%dMtb/dt'] = df.groupby('Unique ID')['Mtb area'].pct_change() * 100

# Handling the first NaN value
df['%dMtb/dt'].fillna(0, inplace=True)


In [51]:
# Ensure that 'Time (hours)' is sorted within each 'Unique ID'
df.sort_values(by=['Unique ID', 'Time (hours)'], inplace=True)

# Calculate the total change in 'Mtb area' for each 'Unique ID'
total_change = df.groupby('Unique ID')['Mtb area'].agg(lambda x: x.iloc[-1] - x.iloc[0])

# Convert the Series to a DataFrame and reset index
total_change_df = total_change.reset_index()
total_change_df.rename(columns={'Mtb area': 'dMtb'}, inplace=True)

# Merge the total change back into the original DataFrame
df = df.merge(total_change_df, on='Unique ID', how='left')

In [54]:
import pandas as pd

# Assuming df is your DataFrame
# Ensure that 'Time (hours)' is sorted within each 'Unique ID'
df.sort_values(by=['Unique ID', 'Time (hours)'], inplace=True)

# Calculate the percentage change in 'Mtb area' for each 'Unique ID'
percentage_change = df.groupby('Unique ID')['Mtb area'].agg(lambda x: ((x.iloc[-1] - x.iloc[0]) / x.iloc[0]) * 100 if x.iloc[0] != 0 else 0)

# Convert the Series to a DataFrame
percentage_change_df = percentage_change.reset_index()
percentage_change_df.rename(columns={'Mtb area': '%dMtb'}, inplace=True)

# Merge the percentage change back into the original DataFrame
df = df.merge(percentage_change_df, on='Unique ID', how='left')

# Now df contains a new column 'Percentage Change in Mtb area' for each row


In [63]:
# Ensure that 'Time (hours)' is sorted within each 'Unique ID'
df.sort_values(by=['Unique ID', 'Time (hours)'], inplace=True)

# Function to apply to each group
def process_group(group):
    # Infection Status
    group['Infection Status'] = (group['Mtb area'] >= 1.92).any()

    # Initial Infection Status
    group['Initial Infection Status'] = group.iloc[:5]['Mtb area'].ge(1.92).any()

    # Final Infection Status
    group['Final Infection Status'] = group.iloc[-5:]['Mtb area'].ge(1.92).any()

    return group

# Apply the function to each group
df = df.groupby('Unique ID').apply(process_group)

In [89]:
# List of columns in the desired order
desired_order = ['Time (hours)', 'Mtb Area', 'dMtb', 'Strain', 'Compound', 'Concentration', 'Unique ID', 'Acquisition ID', 'Cell ID']

# Append the Mphi-related properties (assuming their names start with 'Mphi')
mphi_columns = [col for col in df.columns if col.startswith('Mphi')]
desired_order.extend(mphi_columns)

# Append the rest of the columns, excluding those already specified and the infection status columns
remaining_columns = [col for col in df.columns if col not in desired_order and 'Infection Status' not in col]
desired_order.extend(remaining_columns)

# Finally, append the infection status columns
infection_status_columns = [col for col in df.columns if 'Infection Status' in col]
desired_order.extend(infection_status_columns)

# Reorder the DataFrame columns
df = df[desired_order]

In [91]:
df.keys()

Index(['Time (hours)', 'Mtb Area', 'dMtb', 'Strain', 'Compound',
       'Concentration', 'Unique ID', 'Acquisition ID', 'Cell ID', 'Mphi Area',
       'Mphi GFP', 'x', 'y', 'Eccentricity', 'MSD', 'Technical Replicate',
       'Strain/Compound', 'dMtb/dt', '%dMtb/dt', '%dMtb', 'Mean Mtb Intensity',
       'Thresholded Mean Mtb Intensity', 'Infection Status',
       'Initial Infection Status', 'Final Infection Status'],
      dtype='object')

In [90]:
df

Time (hours)   Mtb Area        dMtb Strain Compound  \
Unique ID                                                               
1.3.5     0                  0  46.797680  136.772588     WT     CTRL   
          1                  1  48.719647  136.772588     WT     CTRL   
          2                  2  52.206007  136.772588     WT     CTRL   
          3                  3  50.552221  136.772588     WT     CTRL   
          4                  4  54.463202  136.772588     WT     CTRL   
...                        ...        ...         ...    ...      ...   
996.4.5   296992            70   0.000000   -3.888632     WT     CTRL   
          296993            71   0.000000   -3.888632     WT     CTRL   
          296994            72   0.000000   -3.888632     WT     CTRL   
          296995            73   0.000000   -3.888632     WT     CTRL   
          296996            74   0.000000   -3.888632     WT     CTRL   

                 Concentration Unique ID Acquisition ID  Cell ID  Mphi Area  \
Unique ID                                                                     
1.3.5     0                EC0     1.3.5         (3, 5)        1    29567.0   
          1                EC0     1.3.5         (3, 5)        1    26181.0   
          2                EC0     1.3.5         (3, 5)        1    26043.0   
          3                EC0     1.3.5         (3, 5)        1    25605.0   
          4                EC0     1.3.5         (3, 5)        1    26430.0   
...                        ...       ...            ...      ...        ...   
996.4.5   296992           EC0   996.4.5         (4, 5)      996    32533.0   
          296993           EC0   996.4.5         (4, 5)      996    31403.0   
          296994           EC0   996.4.5         (4, 5)      996    33239.0   
          296995           EC0   996.4.5         (4, 5)      996    42445.0   
          296996           EC0   996.4.5         (4, 5)      996    35463.0   

                  ...  Technical Replicate  Strain/Compound   dMtb/dt  \
Unique ID         ...                                                   
1.3.5     0       ...                    1             CTRL  0.000000   
          1       ...                    1             CTRL  1.921968   
          2       ...                    1             CTRL  3.486360   
          3       ...                    1             CTRL -1.653786   
          4       ...                    1             CTRL  3.910981   
...               ...                  ...              ...       ...   
996.4.5   296992  ...                    2             CTRL  0.000000   
          296993  ...                    2             CTRL  0.000000   
          296994  ...                    2             CTRL  0.000000   
          296995  ...                    2             CTRL  0.000000   
          296996  ...                    2             CTRL  0.000000   

                  %dMtb/dt      %dMtb  Mean Mtb Intensity  \
Unique ID                                                   
1.3.5     0       0.000000  292.26361          473.168976   
          1       4.106972  292.26361          498.701813   
          2       7.155963  292.26361          502.468353   
          3      -3.167808  292.26361          501.905182   
          4       7.736516  292.26361          502.564423   
...                    ...        ...                 ...   
996.4.5   296992  0.000000 -100.00000          385.325806   
          296993  0.000000 -100.00000          384.643707   
          296994  0.000000 -100.00000          384.512726   
          296995  0.000000 -100.00000          383.335663   
          296996  0.000000 -100.00000          384.480499   

                 Thresholded Mean Mtb Intensity  Infection Status  \
Unique ID                                                           
1.3.5     0                          167.325424              True   
          1                          202.661972              True   
          2         

In [92]:
df.to_pickle('/mnt/DATA/macrohet/results/preliminary_sc_measures/sc_dfs/sc_df_GT_70_area.pkl')